In [1]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, ElasticNet, Lasso, LassoCV
from sklearn.metrics import mean_squared_error
ad = pd.read_csv("Hitters.csv")
df = ad.copy()
df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


In [2]:
df


,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,497,127,7,65,48,37,5,2703,806,32,379,311,138,N,E,325,9,3,700.0,N
318,492,136,5,76,50,94,12,5511,1511,39,897,451,875,A,E,313,381,20,875.0,A
319,475,126,3,61,43,52,6,1700,433,7,217,93,146,A,W,37,113,7,385.0,A
320,573,144,9,85,60,78,8,3198,857,97,470,420,332,A,E,1314,131,12,960.0,A


In [3]:
df.describe().T



,count,mean,std,min,25%,50%,75%,max
AtBat,322.0,380.928571,153.404981,16.0,255.25,379.5,512.00,687.0
Hits,322.0,101.024845,46.454741,1.0,64.00,96.0,137.00,238.0
HmRun,322.0,10.770186,8.709037,0.0,4.00,8.0,16.00,40.0
Runs,322.0,50.909938,26.024095,0.0,30.25,48.0,69.00,130.0
RBI,322.0,48.027950,26.166895,0.0,28.00,44.0,64.75,121.0
Walks,322.0,38.742236,21.639327,0.0,22.00,35.0,53.00,105.0
Years,322.0,7.444099,4.926087,1.0,4.00,6.0,11.00,24.0
CAtBat,322.0,2648.683230,2324.205870,19.0,816.75,1928.0,3924.25,14053.0
CHits,322.0,717.571429,654.472627,4.0,209.00,508.0,1059.25,4256.0
CHmRun,322.0,69.490683,86.266061,0.0,14.00,37.5,90.00,548.0


In [4]:
df.shape

(322, 20)

In [5]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   AtBat      322 non-null    int64  
 1   Hits       322 non-null    int64  
 2   HmRun      322 non-null    int64  
 3   Runs       322 non-null    int64  
 4   RBI        322 non-null    int64  
 5   Walks      322 non-null    int64  
 6   Years      322 non-null    int64  
 7   CAtBat     322 non-null    int64  
 8   CHits      322 non-null    int64  
 9   CHmRun     322 non-null    int64  
 10  CRuns      322 non-null    int64  
 11  CRBI       322 non-null    int64  
 12  CWalks     322 non-null    int64  
 13  League     322 non-null    object 
 14  Division   322 non-null    object 
 15  PutOuts    322 non-null    int64  
 16  Assists    322 non-null    int64  
 17  Errors     322 non-null    int64  
 18  Salary     263 non-null    float64
 19  NewLeague  322 non-null    object 
dtypes: float64

In [6]:
df.dtypes


AtBat          int64
Hits           int64
HmRun          int64
Runs           int64
RBI            int64
Walks          int64
Years          int64
CAtBat         int64
CHits          int64
CHmRun         int64
CRuns          int64
CRBI           int64
CWalks         int64
League        object
Division      object
PutOuts        int64
Assists        int64
Errors         int64
Salary       float64
NewLeague     object
dtype: object

In [7]:
df.isnull().sum()

AtBat         0
Hits          0
HmRun         0
Runs          0
RBI           0
Walks         0
Years         0
CAtBat        0
CHits         0
CHmRun        0
CRuns         0
CRBI          0
CWalks        0
League        0
Division      0
PutOuts       0
Assists       0
Errors        0
Salary       59
NewLeague     0
dtype: int64

In [8]:
#object tipini kategorik tipe dönüştürme işlemi
df.League = pd.Categorical(df.League)
df.Division = pd.Categorical(df.Division)
df.NewLeague = pd.Categorical(df.NewLeague)


In [9]:
kat_df = df.select_dtypes(include=["category"])
kat_df.head()

,League,Division,NewLeague
0,A,E,A
1,N,W,N
2,A,W,A
3,N,E,N
4,N,E,N


In [10]:
df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


In [11]:
num_df = df.select_dtypes(include = ["float64","int64"])

In [12]:
kat_df = pd.get_dummies(kat_df, columns = ['League', 'Division', 'NewLeague'], drop_first = True)

In [13]:
kat_df.head()

,League_N,Division_W,NewLeague_N
0,0,0,0
1,1,1,1
2,0,1,0
3,1,0,1
4,1,0,1


In [14]:
# Değişkenlerdeki eşşiz gözlem sayılarına erişim sağlandı.
df.nunique()

AtBat        247
Hits         144
HmRun         36
Runs          96
RBI          103
Walks         89
Years         22
CAtBat       314
CHits        288
CHmRun       146
CRuns        261
CRBI         262
CWalks       248
League         2
Division       2
PutOuts      232
Assists      161
Errors        29
Salary       150
NewLeague      2
dtype: int64

In [15]:
num_df.corr()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,Salary
AtBat,1.000000,0.967939,0.592198,0.913060,0.820539,0.669845,0.047372,0.235526,0.252717,0.236659,0.266534,0.244053,0.166123,0.317550,0.353824,0.352117,0.394771
Hits,0.967939,1.000000,0.562158,0.922187,0.811073,0.641211,0.044767,0.227565,0.255815,0.202712,0.261787,0.232005,0.151818,0.310673,0.320455,0.310038,0.438675
HmRun,0.592198,0.562158,1.000000,0.650988,0.855122,0.481014,0.116318,0.221882,0.220627,0.493227,0.262361,0.351979,0.233154,0.282923,-0.106329,0.039318,0.343028
Runs,0.913060,0.922187,0.650988,1.000000,0.798206,0.732213,0.004541,0.186497,0.204830,0.227913,0.250556,0.205976,0.182168,0.279347,0.220567,0.240475,0.419859
RBI,0.820539,0.811073,0.855122,0.798206,1.000000,0.615997,0.146168,0.294688,0.308201,0.441771,0.323285,0.393184,0.250914,0.343186,0.106591,0.193370,0.449457
Walks,0.669845,0.641211,0.481014,0.732213,0.615997,1.000000,0.136475,0.277175,0.280671,0.332473,0.338478,0.308631,0.424507,0.299515,0.149656,0.129382,0.443867
Years,0.047372,0.044767,0.116318,0.004541,0.146168,0.136475,1.000000,0.920289,0.903631,0.726872,0.882877,0.868812,0.838533,-0.004684,-0.080638,-0.162140,0.400657
CAtBat,0.235526,0.227565,0.221882,0.186497,0.294688,0.277175,0.920289,1.000000,0.995063,0.798836,0.983345,0.949219,0.906501,0.062283,0.002038,-0.066922,0.526135
CHits,0.252717,0.255815,0.220627,0.204830,0.308201,0.280671,0.903631,0.995063,1.000000,0.783306,0.984609,0.945141,0.890954,0.076547,-0.002523,-0.062756,0.548910
CHmRun,0.236659,0.202712,0.493227,0.227913,0.441771,0.332473,0.726872,0.798836,0.783306,1.000000,0.820243,0.929484,0.799983,0.112724,-0.158511,-0.138115,0.524931


In [16]:
# Eksik gözlemler KNN Algoritması ile dolduruldu.
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors = 5)
df_filled = imputer.fit_transform(num_df)

In [17]:
# Tekrar sorgulandı eksik değer var mı diye hepsinin KNN algoritması ile doldurulduğu gözlemlendi.
num_df = pd.DataFrame(df_filled,columns = num_df.columns)
num_df.isnull().sum()

AtBat      0
Hits       0
HmRun      0
Runs       0
RBI        0
Walks      0
Years      0
CAtBat     0
CHits      0
CHmRun     0
CRuns      0
CRBI       0
CWalks     0
PutOuts    0
Assists    0
Errors     0
Salary     0
dtype: int64

In [18]:
y = num_df["Salary"]
X = num_df.drop('Salary', axis=1)


from sklearn import preprocessing
preprocessing.normalize(X)

X = pd.DataFrame(preprocessing.normalize(X))


In [19]:
df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


In [20]:
X = pd.concat([X,kat_df], axis=1)

In [21]:
df.shape

(322, 20)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state= 46)

**Dogrusal Regresyon**

In [25]:
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, ElasticNet, Lasso, LassoCV

In [26]:
reg_model = LinearRegression()
reg_model.fit(X_train, y_train)
y_pred = reg_model.predict(X_train)
reg_model_egitim_hata = np.sqrt(mean_squared_error(y_train, y_pred))
reg_model_egitim_hata

280.41887062877106

In [27]:
y_pred = reg_model.predict(X_test)
reg_model_test_hata = np.sqrt(mean_squared_error(y_test, y_pred))
reg_model_test_hata

358.72530266671487

In [43]:
reg_model.intercept_

369.9255374614457

In [44]:
reg_model.coef_

array([  1416.76435561,  -5179.63830038,   8093.35874865,  -4277.44984141,
        -7831.59288502,    679.85013361, -17179.06044742,  -2941.38071055,
         7656.89417196,  -1943.45751703,   5073.7363056 ,   6324.79835896,
          459.31062712,    411.45722917,    961.46826872,  -2504.88238673,
          -76.56473008,    -80.50980602,     58.00139662])

**Ridge Regresyon**

In [28]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import model_selection
import matplotlib.pyplot as plt
from sklearn.linear_model import RidgeCV

In [29]:
ridreg = Ridge()
ridreg_model = ridreg.fit(X_train, y_train)
ridreg_y_pred = ridreg_model.predict(X_test)
ridreg_rmse = np.sqrt(mean_squared_error(y_test,ridreg_y_pred))
ridreg_rmse

317.81491232804996

In [30]:
#ridge hiperparametre optimizasyonu

In [31]:
alpha = [0.1,0.01,0.001,0.2,0.3,0.5,0.8,0.9,1]
ridreg_cv = RidgeCV(alphas = alpha, scoring = "neg_mean_squared_error", cv = 10, normalize = True)
ridreg_cv.fit(X_train, y_train)
print(ridreg_cv.alpha_)
#Final Model 
ridreg_tuned = Ridge(alpha = ridreg_cv.alpha_).fit(X_train,y_train)
ridreg_tuned_y_pred = ridreg_tuned.predict(X_test)
ridreg_tuned_rmse = np.sqrt(mean_squared_error(y_test,ridreg_tuned_y_pred))
print(ridreg_tuned_rmse)

0.001
302.1256058781207


In [32]:
#ridge egitim hatasi

In [33]:
y_pred = ridreg_model.predict(X_train)
ridge_model_egitim_hata = np.sqrt(mean_squared_error(y_train, y_pred))
ridge_model_egitim_hata

345.7354370083615

In [34]:
#ridge test hatasi

In [35]:
y_pred = ridreg_model.predict(X_test)
ridge_model_test_hata = np.sqrt(mean_squared_error(y_test, y_pred))
ridge_model_test_hata

317.81491232804996

In [36]:
#Lasso Regresyon

In [37]:
lasreg = Lasso()
lasreg_model = lasreg.fit(X_train,y_train)
lasreg_y_pred = lasreg_model.predict(X_test)
lasreg_rmse = np.sqrt(mean_squared_error(y_test,lasreg_y_pred))
lasreg_rmse

282.0604799479762

In [38]:
#Lasso Regresyon hiperparametre optimizasyonu

In [39]:
alpha = [0.1,0.01,0.001,0.2,0.3,0.5,0.8,0.9,1]
lasso_cv = LassoCV(alphas = alpha, cv = 10, normalize = True)
lasso_cv.fit(X_train, y_train)
print(lasso_cv.alpha_)
#Final Model 
lasso_tuned = Lasso(alpha = lasso_cv.alpha_).fit(X_train,y_train)
lasso_tuned_y_pred = lasso_tuned.predict(X_test)
lasso_tuned_rmse = np.sqrt(mean_squared_error(y_test,lasso_tuned_y_pred))
print(lasso_tuned_rmse)

0.01
328.9297874609108


In [40]:
#elasticnet regresyon

In [41]:
enet = ElasticNet()
enet_model = enet.fit(X_train,y_train)
enet_y_pred = enet_model.predict(X_test)
enet_rmse = np.sqrt(mean_squared_error(y_test,enet_y_pred))
enet_rmse

375.4979788241389

In [42]:
enet_params = {"l1_ratio": [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1],
              "alpha":[0.1,0.01,0.001,0.2,0.3,0.5,0.8,0.9,1]}

enet_model = ElasticNet().fit(X_train,y_train)
enet_cv = GridSearchCV(enet_model, enet_params, cv = 10).fit(X, y)
print(enet_cv.best_params_)

#Final Model 
enet_tuned = ElasticNet(**enet_cv.best_params_).fit(X_train,y_train)
enet_tuned_y_pred = enet_tuned.predict(X_test)
enet_tuned_rmse = np.sqrt(mean_squared_error(y_test,enet_tuned_y_pred))
print(enet_tuned_rmse)

{'alpha': 0.1, 'l1_ratio': 1}
280.9958696521642
